![EDA](vis/v1.jpg)
---
`En este cuadernillo se realiza el análisis exploratorio del dataset airquality.arrow`

#### **Importación de librerías** ####
---

*Para este cuaderno usaremos las siguientes librerías: **os, polars, pandas, datetime, numpy, pyarrow y sodapy**.*

In [363]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import date, time
import numpy as np
import pyarrow.parquet as pq
from sodapy import Socrata

#### **Web Scraping / Extracción de los datos** ####
---

*Para este cuaderno usaremos la api de datos oficiales de la ciudad de Nueva York para abrir las cifras oficiales de la calidad del aire.*

1. Usamos la biblioteca sodapy que nos proporciona una interfaz de cliente para interactuar con la API de Socrata, una plataforma de datos abiertos usada por la ciudad de Nueva York e inicializamos el cliente para interactuar con el portal de datos abiertos de la ciudad. En este caso no necesitamos autenticación por lo que se determina el argumento en None.

2. Creamos una variable que cuente la cantidad total de datos que tiene la api, para traer toda la información que tenga hasta la última actualización

3. Obtenemos los datos de la API estableciendo un id para el ingreso y se establece un limite usando la variable total_filas que es la cantidad total de datos que tiene la API.

4. Convertirmos los datos a un dataframe de Pandas.

In [364]:
# Abrimos el dataset directamente de la API
client = Socrata("data.cityofnewyork.us", None)

# Obtenemos la cantidad total de filas en el conjunto de datos.
total_filas = int(client.get("c3uy-2p5r", select="COUNT(*)")[0]['COUNT'])

# Obtener los datos del conjunto de datos.
results = client.get("c3uy-2p5r", limit=total_filas)

# Convert to pandas DataFrame (16218, 12)
dfAirQuality = pd.DataFrame.from_records(results)

### **Análisis Exploratorio Inicial** ###
---

*Vamos a realizar una **exploración de los datos** (EDA) antes de hacer transformación de los mismos, esto con el fin de **conocer las columnas** y determinar que campos son valiosos para nuestro análisis y así tener claro que pasos realizaremos en el ETL.*

**Nota:**: *Posterior al ETL realizaremos también un EDA enfocado en los datos y una descripción estadística.*

#### **Revisión de la extracción de los datos** ####

*Revisamos que los datos se cargaron adecuadamente en nuestro dataframe **dfAirQuality**:*

1. *Usamos **head** para previsualizar los primeros registros del Dataframe.*
2. *Podemos observar que se cargaron adecuadamente los datos, por ahora no vemos anomalías en los mismos.*

In [365]:
dfAirQuality.head(5)

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,172653,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2011,2010-12-01T00:00:00.000,25.30
1,172585,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2009,2008-12-01T00:00:00.000,26.93
2,336637,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,204,East New York,Annual Average 2015,2015-01-01T00:00:00.000,19.09
3,336622,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,103,Fordham - Bronx Pk,Annual Average 2015,2015-01-01T00:00:00.000,19.76
4,172582,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104,Pelham - Throgs Neck,Annual Average 2009,2008-12-01T00:00:00.000,22.83


#### **Revisión de la forma de los datos** ####

1. *Usamos **shape** obtener el número de filas y columnas del dataframe.*
2. *Observamos que actualmente tenemos 16.218 registros y 11 columnas*

In [366]:
dfAirQuality.shape

(16218, 11)

#### **Exploración de las columnas** ####

*Vamos a revisar qué columnas existen en el dataframe y de qué tipo son:*

1. *Usamos el método **info()** para listar las columnas, su valores no nulos y sus tipos.*
2. *Observamos que no tenemos registros nulos y que todas la variables son de tipo object*

In [367]:
dfAirQuality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16218 entries, 0 to 16217
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   unique_id       16218 non-null  object
 1   indicator_id    16218 non-null  object
 2   name            16218 non-null  object
 3   measure         16218 non-null  object
 4   measure_info    16218 non-null  object
 5   geo_type_name   16218 non-null  object
 6   geo_join_id     16218 non-null  object
 7   geo_place_name  16218 non-null  object
 8   time_period     16218 non-null  object
 9   start_date      16218 non-null  object
 10  data_value      16218 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


##### Revisamos cada una de las columnas

*Con el fin de explorar a nivel de detalle el dataset obtenido realizamos un análisis por columna con el fin de tomar decisiones sobre la ideonidad de los datos y depurar información innecesaria.*

**Columna unique_id**

1. *Según el diccionario de datos de la fuente oficial esta columna representa los identificadores únicos de cada registro del que se tiene información sobre la calidad del aire.*
2. *Revisamos que efectivamente esta tabla cuente con identificadores **únicos**.*
3. *Observamos que esta columna cuenta con los identificadores de cada registro, sin embargo al ser id que no tienen relación con otras tablas, podemos eliminar esta columna*

In [368]:
# Creamos la distribución de frecuencias para observar si hay valores repetidos.
dfAirQuality.unique_id.value_counts().sort_index()

unique_id
121644    1
121645    1
121646    1
121647    1
121648    1
         ..
799864    1
799865    1
799866    1
799867    1
799868    1
Name: count, Length: 16218, dtype: int64

In [369]:
# Eliminamos la columna
dfAirQuality.drop('unique_id', axis=1, inplace=True)

**Columna start_date**

1. *Según el diccionario de datos de la fuente oficial esta columna representa el inicio del periodo de tiempo en el que se midió este indicador.*
2. *Creamos una tabla de frecuencia para revisar los valores de esta columna y tomar decisiones.*
3. *Para efectos de este análisis solo tomaremos los indicadores de los últimos tres años (2019 a 2021) en los que se tomaron los datos.*
4. *Observamos que no es necesario tomar la hora, ya que todas las medidas se tomaron a las 12 horas y no es relevante para este análisis.*

In [370]:
# Creamos una tabla de frecuencias
dfAirQuality.start_date.value_counts().sort_index()

start_date
2005-01-01T00:00:00.000    887
2008-12-01T00:00:00.000    564
2009-01-01T00:00:00.000    480
2009-06-01T00:00:00.000    564
2009-12-01T00:00:00.000    564
2010-06-01T00:00:00.000    423
2010-12-01T00:00:00.000    564
2011-01-01T00:00:00.000    214
2011-06-01T00:00:00.000    423
2011-12-01T00:00:00.000    564
2012-01-02T00:00:00.000    480
2012-06-01T00:00:00.000    423
2012-12-01T00:00:00.000    564
2013-01-01T00:00:00.000    144
2013-06-01T00:00:00.000    423
2013-12-01T00:00:00.000    564
2014-01-01T00:00:00.000     96
2014-06-01T00:00:00.000    423
2014-12-01T00:00:00.000    282
2015-01-01T00:00:00.000    906
2015-06-01T00:00:00.000    423
2015-12-01T00:00:00.000    282
2015-12-31T00:00:00.000    282
2016-01-01T00:00:00.000    321
2016-06-01T00:00:00.000    423
2016-12-01T00:00:00.000    282
2017-01-01T00:00:00.000    282
2017-06-01T00:00:00.000    423
2017-12-01T00:00:00.000    282
2018-01-01T00:00:00.000    282
2018-06-01T00:00:00.000    423
2018-12-01T00:00:00.000    2

In [371]:
# Convertimos el tipo de esta columna a datetime.

dfAirQuality["start_date"] = pd.to_datetime(dfAirQuality["start_date"])

In [372]:
# Filtramos el dataframe para incluir solo los registros de los últimos 3 años y eliminamos la hora, ya que no es relevante.

dfAirQuality = dfAirQuality[(dfAirQuality['start_date'].dt.year >= 2019)]
dfAirQuality['start_date'] = dfAirQuality['start_date'].dt.date

In [373]:
# Verificamos los cambios realizados

dfAirQuality.start_date.value_counts()

start_date
2020-06-01    423
2021-06-01    423
2019-06-01    423
2020-01-01    282
2021-01-01    282
2019-01-01    282
2020-12-01    282
2019-12-01    282
Name: count, dtype: int64

**Columna indicator_id y name**

1. *Según el diccionario de datos de la fuente oficial estas columna representa los identificadores únicos y los nombres de las mediciones de cada registro del que se tiene información sobre la calidad del aire.*
2. *Si comparamos las dos columnas observamos que presentan la misma distribución de los datos.*
3. *Cambiamos los valores a español*
4. *Eliminamos la columna indicator_id*

In [374]:
# Comparamos los valores de indicator_id
dfAirQuality.indicator_id.value_counts()

indicator_id
375    1128
365    1128
386     423
Name: count, dtype: int64

In [375]:
# Comparamos los valores de name
dfAirQuality.name.value_counts()

name
Nitrogen dioxide (NO2)     1128
Fine particles (PM 2.5)    1128
Ozone (O3)                  423
Name: count, dtype: int64

In [376]:
# Mapeamos los valores y los reemplazamos

mapeo_name = {
    "Nitrogen dioxide (NO2)": "NO2",
    "Fine particles (PM 2.5)": "PM 2.5",
    "Ozone (O3)": "O3"
}

dfAirQuality["name"] = dfAirQuality["name"].replace(mapeo_name)

In [377]:
# Eliminamos la columna indicator id
dfAirQuality.drop("indicator_id", axis=1, inplace=True)

**Columna measure**

1. *Según el diccionario de datos de la fuente oficial estas columna representa el valor que toma cada indicador, es decir como se calcula este valor*
2. *Si contamos los valores que toma esta columna, observamos que todas se calculan como la media*
3. *Eliminamos esta columna, ya que no nos aporta información.*

In [378]:
dfAirQuality.measure.value_counts()

measure
Mean    2679
Name: count, dtype: int64

In [379]:
# Eliminamos la columna indicator id
dfAirQuality.drop("measure", axis=1, inplace=True)

**Columna measure_info y data_value**

1. *Según el diccionario de datos de la fuente oficial estas columna representa las unidades de medida que se utilizaron para informar las medidas que se tomaron*
2. *Si contamos los valores que toma esta columna, observamos que existen dos medidas de referencia: partes por billon y microgramos por metro cúbico*
3. *Vamos a normalizar todos los datos a una sola medida, por lo que revisamos en internet como es el proceso de conversión, para revisar las formulas que se usaron puede seguir el siguiente [enlace](https://www.youtube.com/watch?v=lTqZV1xaZUg&ab_channel=JoseLoboGomez).*
4. *Convertimos la columna data_value a valores de tipo float.*
5. *Se definen las masas molares y las medias de temperatura de la ciudad de Nueva York de acuerdo al mes en el que se tomaron las medidas*
6. *Se crean los valores usando la función y se eliminan las columnas que ya no se necesitan*


In [380]:
dfAirQuality.measure_info.value_counts()

measure_info
ppb       1551
mcg/m3    1128
Name: count, dtype: int64

In [381]:
dfAirQuality.start_date.value_counts()

start_date
2020-06-01    423
2021-06-01    423
2019-06-01    423
2020-01-01    282
2021-01-01    282
2019-01-01    282
2020-12-01    282
2019-12-01    282
Name: count, dtype: int64

In [382]:
# Convertimos el tipo de dato de la columna data_value a float.
dfAirQuality["data_value"] = dfAirQuality["data_value"].astype(float)

In [383]:
# Definimos las masas molares des los gases contaminantes teniendo en cuenta los pesos atómicos que los componen.
masaOzono = 3 * 16
masaNitrogeno = 14 + 2 * 16
masaPM = 3.625 #Este es un estándar para el material particulado.

# Mapeamos estos valores teniendo en cuenta los nombres de los componentes para luego aplicarlos a la función
masa_map = {
    "O3": masaOzono,
    "NO2": masaNitrogeno,
    "PM 2.5": masaPM,
}

In [384]:
# Definimos los promedios de temperatura teniendo en cuenta los datos historicos de temperatura por meses
temEne = 0.5
temJun = 22.5
temDic = 3.5

# Creamos una función que nos permita identificar el mes y asignarle un valor de temperatura
def obtener_tem(mes):
    if mes == 1:  # Enero
        return temEne
    elif mes == 6:  # Junio
        return temJun
    elif mes == 12:  # Diciembre
        return temDic
    else:
        return None
    
# Creamos una nueva columna que contenga la temperatura promedio para ese mes.
dfAirQuality["tem"] = pd.to_datetime(dfAirQuality["start_date"]).dt.month.apply(obtener_tem)


In [385]:
# Creamos una función que nos permite hacer la conversión tomando como parámetros la medida en ppb, la temperatura y la masa.
def calcular_mcg(ppb, tem, masa):
    kelvin = tem + 273.15
    vol = 0.082 * kelvin
    res = (ppb * 1000 * masa * 10 **6) / (10 **9 * vol)
    return round(res,2)

In [386]:
# Aplicamos la función de conversión para crear la nueva columna, solo convertiremos aquellos valores que se encuentren en ppb.
dfAirQuality["mcg/m3"] = dfAirQuality.apply(lambda row: calcular_mcg(row["data_value"], row["tem"], masa_map[row["name"]])
                               if row["measure_info"] == "ppb" else row["data_value"], axis=1)

In [387]:
# Eliminamos las columnas que measure_info, data_value y tem
dfAirQuality.drop("measure_info", axis=1, inplace=True)
dfAirQuality.drop("data_value", axis=1, inplace=True)
dfAirQuality.drop("tem", axis=1, inplace=True)

**Columna time_period**

1. *Según el diccionario de datos de la fuente oficial esta columna representa el periodo de tiempo en el que se tomaron los datos, esto puede ser anual o semestral (Invierno y Verano).*
2. *Normalizamos los valores de las variables para que solo sean: Año, Invierno, Verano.*

In [388]:
dfAirQuality.time_period.value_counts()

time_period
Summer 2020            423
Summer 2021            423
Summer 2019            423
Annual Average 2020    282
Annual Average 2021    282
Annual Average 2019    282
Winter 2020-21         282
Winter 2019-20         282
Name: count, dtype: int64

In [389]:
# Mapeamos los valores y los reemplazamos

mapeo_time_period = {
    "Summer 2020": "Verano",
    "Summer 2021": "Verano",
    "Summer 2019": "Verano",
    "Annual Average 2020": "Año",
    "Annual Average 2021": "Año",
    "Annual Average 2019": "Año",
    "Winter 2020-21": "Invierno",
    "Winter 2019-20": "Invierno",
}

dfAirQuality["time_period"] = dfAirQuality["time_period"].replace(mapeo_time_period)

**Renombramos las columnas**

1. *Renombramos las columnas para mejorar la comprensión de los datos.*

In [390]:
dfAirQuality = dfAirQuality.rename(columns={"name": "Contaminante", "geo_type_name": "Division", "geo_join_id": "IdLoc",
                                            "geo_place_name": "NombreLugar",  "time_period": "PeriodoTiempo",  "start_date": "FechaInicio"})
dfAirQuality.head(3)

,Contaminante,Division,IdLoc,NombreLugar,PeriodoTiempo,FechaInicio,mcg/m3
5,NO2,UHF34,104,Pelham - Throgs Neck,Año,2020-01-01,33.19
12,NO2,UHF42,106,High Bridge - Morrisania,Año,2021-01-01,36.98
18,NO2,UHF42,410,Rockaways,Año,2021-01-01,23.39


**Columna Division, IdLoc, NombreLugar**

1. *Según el diccionario de datos de la fuente oficial estas columnas representan la zona sobre la cual se toma la medición, estas mediciones se tomaron en distintos niveles (Citywide = A nivel de ciudad), (Borough = A nivel de cada Distrito (Borough)), (UHF 34 y UHF 42 = A nivel de Zonas de planificación de servicios de salud) y (CD =Distritos comunitarios)*
2. *Observamos la distribución de datos que tenemos por división.*
3. *Vamos a separar los datos de acuerdo a la división de la ciudad, así en el análisis podremos analizar los datos a diferentes escalas.*

In [391]:
dfAirQuality.Division.value_counts()

Division
CD          1121
UHF42        798
UHF34        646
Borough       95
Citywide      19
Name: count, dtype: int64

**Dataframe AirQualityNewYorkCity**

1. *Este dataframe contiene los medidas generales de la ciudad en cuanto a niveles de contaminantes en el aire.*
2. *Eliminamos las columnas de localización ya que este dataframe solo se refiere a la ciudad de Nueva York en conjunto.*
3. *Ordenamos por fecha para mejorar la legibilidad de la tabla.*

In [392]:
# Creamos el dataframe con los datos generales de la ciudad.

dfAQNYC = dfAirQuality.loc[dfAirQuality["Division"].str.contains("Citywide")].copy()
dfAQNYC.drop(columns=["Division","IdLoc", "NombreLugar"], axis=1, inplace=True)
dfAQNYC.sort_values(by=["FechaInicio"], inplace=True)
dfAQNYC.head(3)

,Contaminante,PeriodoTiempo,FechaInicio,mcg/m3
15409,NO2,Año,2019-01-01,31.96
15310,PM 2.5,Año,2019-01-01,6.60
15389,NO2,Verano,2019-06-01,24.63


**Dataframe AirQualityBoroughs**

1. *Este dataframe contiene los medidas generales de los 5 boroughs de la ciudad en cuanto a niveles de contaminantes en el aire.*
2. *Eliminamos las columnas que no son necesarias*
3. *Mapeamos los id de los boroughs para que coincidan con las fuentes oficiales*
4. *Hacemos algunos ajustes al dataframe*

In [393]:
# Creamos el dataframe con los datos de cada uno de los 5 boroughs

dfAQBoroughs = dfAirQuality.loc[dfAirQuality["Division"].str.contains("Borough")].copy()
dfAQBoroughs.drop(columns=["Division"], axis=1, inplace=True)
dfAQBoroughs.head(3)

,Contaminante,IdLoc,NombreLugar,PeriodoTiempo,FechaInicio,mcg/m3
133,NO2,1,Bronx,Verano,2020-06-01,24.97
137,NO2,2,Brooklyn,Verano,2020-06-01,20.53
232,NO2,4,Queens,Año,2020-01-01,30.38


In [394]:
# Mapeamos los id de los boroughs para que coincidan con los id oficiales de los boroughs de NYC.

mapeo_borough = {
    "1": 2,
    "2": 3,
    "3": 1,
    "4": 4,
    "5": 5
}

dfAQBoroughs["IdLoc"] = dfAirQuality["IdLoc"].replace(mapeo_borough)

In [395]:
# Creamos un dataframe que contenga los códigos de cada Borough y su nombre
dfBor = dfAQBoroughs[["IdLoc", "NombreLugar"]].drop_duplicates().sort_values(by="IdLoc")
dfBor.rename(columns={"IdLoc": "IdBorough", "NombreLugar": "Nombre"}, inplace=True)
dfBor["IdBorough"] = dfBor["IdBorough"].astype(int)
dfBor


,IdBorough,Nombre
352,1,Manhattan
133,2,Bronx
137,3,Brooklyn
232,4,Queens
6330,5,Staten Island


In [396]:
# Eliminamos la columna nombre lugar, ya que no será necesaria.

dfAQBoroughs.drop(columns=["NombreLugar"], axis=1, inplace=True)

In [397]:
# Reordenamos el dataframe para mejorar la lectura del mismo y cambiamos el nombre de la columna Id

dfAQBoroughs = dfAQBoroughs.reindex(columns=["IdLoc", "FechaInicio","PeriodoTiempo", "Contaminante", "mcg/m3" ])
dfAQBoroughs.rename(columns={"IdLoc": "IdBorough"}, inplace=True)
dfAQBoroughs.sort_values(by=["IdBorough", "FechaInicio"], inplace=True)
dfAQBoroughs["IdBorough"] = dfAQBoroughs["IdBorough"].astype(int)

In [398]:
dfAQBoroughs

,IdBorough,FechaInicio,PeriodoTiempo,Contaminante,mcg/m3
15285,1,2019-01-01,Año,NO2,44.50
15378,1,2019-01-01,Año,PM 2.5,8.31
12122,1,2019-06-01,Verano,O3,51.18
13454,1,2019-06-01,Verano,NO2,37.38
13795,1,2019-06-01,Verano,PM 2.5,9.44
...,...,...,...,...,...
15900,5,2021-01-01,Año,NO2,26.32
16057,5,2021-01-01,Año,PM 2.5,6.15
15842,5,2021-06-01,Verano,O3,58.47
16068,5,2021-06-01,Verano,NO2,14.65


**Dataframes UHF34 y UHF42**

1. *Este dataframe contiene los medidas generales de 34 y 42 zonas de planificación de servicios de salud, que agrupan un conjunto de barrios en cuanto a niveles de contaminantes en el aire.*
2. *Eliminamos las columnas que no son necesarias*
3. *No usaremos estos dataframes para el análisis*

In [399]:
# Creamos el dataframe con los datos de cada uno las 34 zonas de planificación de servicios de salud, sin embargo esta información no la usaremos.
dfAQUHF34 = dfAirQuality.loc[dfAirQuality["Division"].str.contains("UHF34")].copy()
dfAQUHF34.drop(columns=["Division"], axis=1, inplace=True)
dfAQUHF34.head(3)

,Contaminante,IdLoc,NombreLugar,PeriodoTiempo,FechaInicio,mcg/m3
5,NO2,104,Pelham - Throgs Neck,Año,2020-01-01,33.19
40,NO2,301,Washington Heights,Año,2020-01-01,37.06
67,NO2,209,Bensonhurst - Bay Ridge,Año,2020-01-01,30.36


In [400]:
# Creamos el dataframe con los datos de cada una de las 42 zonas de planificación de servicios de salud, sin embargo esta información no la usaremos.
dfAQUHF42 = dfAirQuality.loc[dfAirQuality["Division"].str.contains("UHF42")].copy()
dfAQUHF42.drop(columns=["Division"], axis=1, inplace=True)
dfAQUHF42.head(3)

,Contaminante,IdLoc,NombreLugar,PeriodoTiempo,FechaInicio,mcg/m3
12,NO2,106,High Bridge - Morrisania,Año,2021-01-01,36.98
18,NO2,410,Rockaways,Año,2021-01-01,23.39
43,NO2,107,Hunts Point - Mott Haven,Año,2021-01-01,37.97


**Dataframe Distritos Comunitarios**

1. *Este dataframe contiene los medidas generales de los 59 Distritos Comunitarios que existen en la ciudad que agrupan un conjunto de barrios en cuanto a niveles de contaminantes en el aire.*
2. *Eliminamos las columnas que no son necesarias*
3. *Creamos un dataframe con la información de cada uno de los distritos.*
4. *Hacemos cambios en la estructura del dataframe.*

In [401]:
# Creamos el dataframe con los datos de cada uno de los 59 Distritos Comunitarios
dfAQCD = dfAirQuality.loc[dfAirQuality['Division'].str.contains("CD")].copy()
dfAQCD.drop(columns=["Division"], axis=1, inplace=True)
dfAQCD.head(3)

,Contaminante,IdLoc,NombreLugar,PeriodoTiempo,FechaInicio,mcg/m3
21,NO2,101,Financial District (CD1),Año,2021-01-01,44.30
26,NO2,402,Woodside and Sunnyside (CD2),Año,2021-01-01,41.43
82,NO2,108,Upper East Side (CD8),Año,2021-01-01,41.76


In [402]:
# Creamos un dataframe que contenga los códigos de cada Distrito Comunitario y su nombre, eliminando los parentesis.
dfCD = dfAQCD[["IdLoc", "NombreLugar"]].drop_duplicates().sort_values(by="IdLoc")
dfCD.rename(columns={"IdLoc": "IdCD", "NombreLugar": "Nombre"}, inplace=True)
dfCD['Nombre'] = dfCD['Nombre'].str.replace(r"\(.*\)", "", regex=True)
dfCD["IdCD"] = dfCD["IdCD"].astype(int)

In [403]:
# Eliminamos la columna nombre lugar, ya que no será necesaria.

dfAQCD.drop(columns=["NombreLugar"], axis=1, inplace=True)

In [404]:
# Reordenamos el dataframe para mejorar la lectura del mismo y cambiamos el nombre de la columna Id

dfAQCD = dfAQCD.reindex(columns=["IdLoc", "FechaInicio", "PeriodoTiempo", "Contaminante", "mcg/m3" ])
dfAQCD.rename(columns={"IdLoc": "IdCD"}, inplace=True)
dfAQCD.sort_values(by=["IdCD", "FechaInicio"], inplace=True)
dfAQCD["IdCD"] = dfAQCD["IdCD"].astype(int)

In [407]:
dfAQCD

,IdCD,FechaInicio,PeriodoTiempo,Contaminante,mcg/m3
12788,101,2019-01-01,Año,NO2,49.81
13171,101,2019-01-01,Año,PM 2.5,8.42
12947,101,2019-06-01,Verano,O3,49.14
12951,101,2019-06-01,Verano,NO2,44.13
13346,101,2019-06-01,Verano,PM 2.5,9.69
...,...,...,...,...,...
1594,503,2021-01-01,Año,PM 2.5,6.03
4821,503,2021-01-01,Año,NO2,22.80
4165,503,2021-06-01,Verano,O3,59.85
4176,503,2021-06-01,Verano,PM 2.5,7.58


#### **Exportación de los datos** ####

*Realizamos la exportación de los dataframes a formato .arrow*

1. *Convertimos el dataframe a polars*
2. *Convertimos el dataframe de polar a un objeto arrow*

In [408]:
# Convertimos el dataframe Air Quality New York City
dfAQNYC = pl.from_pandas(dfAQNYC)
arrowAQNYC = dfAQNYC.to_arrow()
pq.write_table(arrowAQNYC, '../datasets/processed/aq_NYC.arrow')

In [409]:
# Convertimos el dataframe Air Quality Boroughs
dfAQBoroughs = pl.from_pandas(dfAQBoroughs)
arrowAQBoroughs = dfAQBoroughs.to_arrow()
pq.write_table(arrowAQBoroughs, '../datasets/processed/aq_boroughs.arrow')

In [410]:
# Convertimos el dataframe Boroughs
dfBor = pl.from_pandas(dfBor)
arrowBor = dfBor.to_arrow()
pq.write_table(arrowBor, '../datasets/processed/boroughs.arrow')

In [411]:
# Convertimos el dataframe Air Quality Community Districts
dfAQCD = pl.from_pandas(dfAQCD)
arrowAQCD = dfAQCD.to_arrow()
pq.write_table(arrowAQCD, '../datasets/processed/aq_cd.arrow')

In [412]:
# Convertimos el dataframe Community Districts
dfCD = pl.from_pandas(dfCD)
arrowCD = dfCD.to_arrow()
pq.write_table(arrowCD, '../datasets/processed/community_districts.arrow')